#### Datathon 2022 - EPI Gijón

In [64]:
## Imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [65]:
## Parsing CSV data

df_amazon = pd.read_csv('data/Amazon.csv')
df_Email = pd.read_csv('data/Email.csv')
df_Facebook = pd.read_csv('data/Facebook.csv')
df_Google = pd.read_csv('data/Google.csv')
df_Instagram = pd.read_csv('data/Instagram.csv')
df_Radio = pd.read_csv('data/Radio.csv')
df_TV = pd.read_csv('data/TV.csv')
df_Youtube = pd.read_csv('data/Youtube.csv')


In [66]:
# Sumar quincenas de Faceboook

df_Facebook['Facebook'] = df_Facebook['Facebook 1 quincena'] + df_Facebook['Facebook 2 quincena']

df_Facebook.drop(columns='Facebook 1 quincena', inplace=True)
df_Facebook.drop(columns='Facebook 2 quincena', inplace=True)
df_Facebook.head(10)

,Date,Facebook
0,01-03-20,0.00
1,01-04-20,73571.23
2,01-05-20,0.00
3,01-06-20,0.00
4,01-07-20,0.00
5,01-08-20,0.00
6,01-09-20,0.00
7,01-10-20,0.00
8,01-11-20,0.00
9,01-12-20,0.00


In [76]:
# Crear un dataframe con todas las columnas
from copy import copy



df = copy(df_amazon)
df['Email'] = df_Email['Email']
df.head(10)

SyntaxError: invalid syntax (2465046629.py, line 7)